### Intro

#### 감성분석?
* 감성 분석은 텍스트의 분류와 연관성 높음 
* 일반적으로 긍정, 부정, (중립) 으로 분류
* 좀 더 많은 category 로 분류하면 텍스트 분류로 생각할 수 있음
* 추천과 연관성 높음
    * 추천에서 널리 쓰는 기반 데이터는 상품평 등의 텍스트
    * 홈페이지 내용, 상품 정보 등에 해당하는 텍스트로 부터 특징을 찾아낼 수 있음

#### 감성분석 모델
* 기본 분석 
* 특정 단어들의 빈도 확인을 통해 문장 분류 : 성능 낮음
* Doc2vec
    * 참조 : Quoc Le and Tomas Mikolov, “Distributed Representations of Sentences and Documents"
    * Word2vec과 같이 단어들을 공간상의 점들로 매핑(임베딩)하면서 각 문장에 포함된 단어들의 위치와 유사한 지역에 해당 문장이 위치하도록 매핑 
    * 각 문장들의 위치가 정해지므로 문장 간 차이 등의 비교 가능
    * 각 문장의 위치를 입력 데이터로 생각해 해당 데이터로부터 문장의 감성 또는 텍스트 예측을 위한 모델 구성
* 주요 모델  
Doc2vec이 MLP기반 모델인데 비해 CNN, RNN, Recursive NN 등의 다양한 네트워크 구성을 통해 문장 정형화 가능  
    * Doc2vec은 일반적인 NN 과 다르게 트레이님으로 만들어진 모델이 테스트에서는 값의 계산만을 통해 얻어지는 것 아님
    * 테스트 과정에서 트레이닝과 유사한 위치 추정 작업이 일어나고 이로 인해 속도 저하 

#### Doc2vec 구현 
단어 및 문장의 임베딩은 Gensim library에 구현되어 이미 널리 사용되고 있음
* 필요 작업 :
    * Gensim 형태에 맞게 데이터 변형
        * 텍스트를 정형화 데이터로 변형 후 사용하지만, Gensim의 해당 함수는 전처리를 내부적으로 수행하여 입력 데이터를 텍스트 형태로 넣음
    * 한글 처리를 위한 형태소 분석
        * 데이터가 아주 많다면 꼭 필요한 것은 아니지만 한글에서는 형태소 분석을 먼저 적용하는 것이 일반적
        * 단어 특성 등을 반영한 데이터 정리(빈도 낮은 단어 제거, 조사 제거 등)
    * 사용 library :
        * Gensim(텍스트 분류)
        * konlpy(한글 처리)

In [1]:
from collections import OrderedDict
from collections import namedtuple
from operator import itemgetter
import re, sys, os.path
from pprint import pprint
import multiprocessing

import pickle
import numpy as np

#### 형태소 분석기
Twitter 에서 공개한 형태소 분석기 사용 (다른 것들로 사용 가능함)

In [2]:
from konlpy.tag import Okt

#### 데이터 로드
Original data are type of np.ndarray  
For example,  
> xs = [[2,3,4,56,6], [4,3,5], ...],  
> ys = [1,0,1,0,1,0]  
> ys_multi = [0,4,3,9,7]  

반면, gensim-doc2vec is very easy to use (By calling corresponding function is sufficient)  
Gensim-doc2vec data format  
> [(['i', 'love', 'you'], '1'), (...), ...]

In [3]:
import naver_movie

data1 = naver_movie.Data() # 긍정, 부정으로 분류 
#data1 = Data('multi') # 1~10점으로 분류 

해당 데이터는 텍스트 데이터를 정형화 한 값들임   
기본적 표현 방식 : 사전 단어들에 대하여 1, ..., N 까지 각 단어에 해당하는 고유 번호를 부여하고,  
각 단어들을 이 고유번호로 대체  
형태소 분석은 문장 내 단어에 데하여 원형 단어와 변형된 값들을 표현 

In [4]:
# load saved data and index information  
(x_train, y_train), (x_test, y_test) = data1.load_data()
temp = data1.get_word_index()

In [5]:
# 각 단어들에 대하여 대칭되는 index 존재 
word2ix, ix2word = temp['word2ix'], temp['ix2word']

Gensim은 텍스트 데이터를 입력으로 넣으면 내부적으로 위의 단어를 index로 변형하는 작업 수행  
인덱스로 표현된 텍스트를 (단어, 인덱스) 매핑 값으로 부터 재변환

In [6]:
train_doc, test_doc = [], []
whole_doc = []
for xis, y1 in zip(x_train, y_train):
    doc1 = []
    for xi in xis:
        doc1.append( ix2word[xi] )
    train_doc.append((doc1, str(y1)))
    whole_doc.append((doc1, str(y1)))
for xis, y1 in zip(x_test, y_test):
    doc1 = []
    for xi in xis:
        doc1.append( ix2word[xi] )
    test_doc.append((doc1, str(y1)))
    whole_doc.append((doc1, str(y1)))

#### Gensim for doc2vec

In [7]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_doc]
tagged_test_docs = [TaggedDocument(d, [c]) for d, c in test_doc]
tagged_whole_docs = [TaggedDocument(d, [c]) for d, c in whole_doc]

from gensim.models import Doc2Vec
import gensim.models.doc2vec

위에서 설명한 것과 같이 Gensim은 test(evaluation) 과정에서 추가 계산 시간이 소요 되어 멀티프로세스 활용(cpu 다중 코어) 

In [8]:
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

#### Model construction
임베딩을 위한 몇 가지의 변형 모델들이 있음  
여기서는 2개 모델(a,b)과 1개의 추가적 세부 조정 모델(a2)을 결합해 두 개의 별도 모델(a+b, a2+b) 을 만들고 최종 결과 확인 
위의 '+' 은 데이터를 concatenate 하는 것으로 두 모델이 별도 구한 임베딩 값(입력 값)을 모두 씀  
> data concatenate
> data1 = [data1a, data1b, ...]
> data2 = [data2a, data2b, ...]
> data = data1 + data2 = [data1a, data1b, ..., data2a, data2b, ...]

In [9]:
model1 = Doc2Vec(dm=1, vector_size=100, window=5, negative=5, hs=0, min_count=2, sample=0, 
                 epochs=20, workers=cores, alpha=0.025, comment='alpha=0.025')
model2 = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=5, negative=5, hs=0, min_count=2, 
                 sample=0, epochs=20, workers=cores)
model3 = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, epochs=20, 
                 workers=cores)
models = [model1, model2, model3]
    
for model in models:
    model.build_vocab(tagged_train_docs)

# data concatenate
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
modela = ConcatenatedDoc2Vec([model1, model2])
modelb = ConcatenatedDoc2Vec([model1, model3])

#### Model training
트레이닝에 필요한 각 문장에 대하여 해당하는 가상공간 상 점으로 표현  
이 점은 문장이 포함한 단어들과 유사한 곳에 위치하게 매핑  

In [10]:
for model in models:
    model.train(tagged_train_docs, total_examples=len(x_train), epochs=model1.epochs)

#pprint(model1.most_similar('연기/Noun'))
train_x = [modela.infer_vector(doc.words) for doc in tagged_train_docs]
train_y = [doc.tags[0] for doc in tagged_train_docs]
    
# 테스트 데이터에 대하여 값 infer를 통해 테스트 문장의 위치 구함 
test_x = [modela.infer_vector(doc.words) for doc in tagged_test_docs]
test_y = [doc.tags[0] for doc in tagged_test_docs]

#### classifier
주어진 위치로 부터 텍스트 분류를 위한 예측 모델을 구성

In [11]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1234)
classifier.fit(train_x, train_y)
print(classifier.score(test_x, test_y))

train_x = [modelb.infer_vector(doc.words) for doc in tagged_train_docs]
train_y = [doc.tags[0] for doc in tagged_train_docs]
test_x = [modelb.infer_vector(doc.words) for doc in tagged_test_docs]
test_y = [doc.tags[0] for doc in tagged_test_docs]

classifier.fit(train_x, train_y)
print(classifier.score(test_x, test_y))

0.69408
0.83014
